<a href="https://colab.research.google.com/github/liamlio/IEEEFraudDetectionComp/blob/master/ieee_fraud_detection_DL_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()
#Upload your kaggle.json api key

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"liamlio","key":"32d0c187f3ae18696e9889dd18b5baf3"}'}

In [0]:
#Moving the kaggle.json file into the .kaggle/ directory
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c ieee-fraud-detection 
#Download the dataset from the kaggle api

 63% 33.0M/52.5M [00:00<00:00, 32.9MB/s]
100% 52.5M/52.5M [00:00<00:00, 67.5MB/s]
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 204MB/s]
 70% 33.0M/47.3M [00:00<00:00, 51.1MB/s]
100% 47.3M/47.3M [00:00<00:00, 87.7MB/s]
  0% 0.00/2.97M [00:00<?, ?B/s]
100% 2.97M/2.97M [00:00<00:00, 199MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 163MB/s]


In [0]:
#unzip all the dataset (this will take a minute to appear in the /content directory)
import zipfile
with zipfile.ZipFile("test_identity.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("test_transaction.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("train_identity.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("train_transaction.csv.zip","r") as zip_ref:
    zip_ref.extractall('/content')

In [0]:
#from comet_ml import Experiment
from fastai.tabular import *
from sklearn import preprocessing
from sklearn.decomposition import PCA, KernelPCA

In [0]:
def load_df(mode = 'train'):
    df = pd.read_csv(f'{mode}_transaction.csv')
    # (590540, 434)
    dfi = pd.read_csv(f'{mode}_identity.csv')
    df =  pd.merge(df, dfi, how='left', on='TransactionID')
    return df


In [0]:
df = load_df()

In [0]:
# OLD STUFF
# dep_var = 'isFraud'
# cat_names = [
#               # Transaction
#               'ProductCD', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain',
#               *[f'card{i}' for i in range(1,7)],
#               *[f'M{i}' for i in range(1,10)],
#               # Identity
#               'DeviceType', 'DeviceInfo', 
#                *[f'id_{i}' for i in range(12,39)],
#              ]
# cont_names = list(set(df.columns)-set(cat_names)-{dep_var})
# df = df.sort_values('TransactionDT')
# #The fact that it's NaN could be a good indicator
# # for col in df.columns:
# #     if col in cat_names:
# #         df[col].fillna('NaN')

# # cat_names = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
# #             'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
# #             'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
# #             'P_emaildomain']
# # cont_names = [i for i in df.keys() if i not in cat_names]

# # Following SafeBox Kernel
# # drop_col = ['isFraud', 'TransactionID', 'TransactionDT', 'V300', 'V309', 'V111', 'C3', 'V124', 'V106', 'V125',
# #             'V315', 'V134', 'V102', 'V123', 'V316', 'V113', 'V136', 'V305', 'V110', 'V299', 'V289', 'V286',
# #             'V318', 'V103', 'V304', 'V116', 'V298', 'V284', 'V293', 'V137', 'V295', 'V301', 'V104', 'V311', 
# #             'V115', 'V109', 'V119', 'V321', 'V114', 'V133', 'V122', 'V319', 'V105', 'V112', 'V118', 'V117', 
# #             'V121', 'V108', 'V135', 'V320', 'V303', 'V297', 'V120']
# # for c in drop_col:
# #     if c in cat_names:
# #         cat_names.remove(c)
# #     else:
# #         cont_names.remove(c)

# procs = [FillMissing, Categorify, Normalize] # Order matters
# path = '';
# metrics = [AUROC()]
# label_cls = CategoryList
# log = False
# y_range = None
# kwargs = {'log': log} if log else {}


In [0]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
    
#     for col in df.columns:
#         col_type = df[col].dtype
        
#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         #else:
#             #df[col] = df[col].astype('category')

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
#         start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df
    #This function seems to reduce results from the neural net

In [0]:
#X_train = reduce_mem_usage(X_train)
#X_test = pd.merge(test_trn[trn_cols], test_id[id_cols], how='left')

#df_train_id = df.pop('TransactionID')
#X_test_id = X_test.pop('TransactionID')

#all_data = df
#Ran into memory problems trying to process both the test and train set together
# :(


In [0]:
# vcols = [f'V{i}' for i in range(1,340)]

# sc = preprocessing.MinMaxScaler()

# pca = PCA(n_components=2) #0.99
# vcol_pca = pca.fit_transform(sc.fit_transform(all_data[vcols].fillna(-1)))

# all_data['_vcol_pca0'] = vcol_pca[:,0]
# all_data['_vcol_pca1'] = vcol_pca[:,1]
# all_data['_vcol_nulls'] = all_data[vcols].isnull().sum(axis=1)

# all_data.drop(vcols, axis=1, inplace=True)

###Higher results without minizing dimensionality of the V features###

In [0]:
#all_data = df
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
all_data['Date'] = all_data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
all_data['_weekday'] = all_data['Date'].dt.dayofweek
all_data['_hour'] = all_data['Date'].dt.hour
all_data['_day'] = all_data['Date'].dt.day

all_data['_weekday'] = all_data['_weekday'].astype(str)
all_data['_hour'] = all_data['_hour'].astype(str)
#all_data['_weekday__hour'] = all_data['_weekday'] + all_data['_hour']

# cnt_day = all_data['_day'].value_counts()
# cnt_day = cnt_day / cnt_day.mean()
# all_data['_count_rate'] = cnt_day #all_data['_day'].map(cnt_day.to_dict())

all_data.drop(['TransactionDT','Date','_day'], axis=1, inplace=True)

In [70]:
# all_data['_weekday'].unique()
# all_data['_hour'].unique()

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '20', '21', '22', '23'], dtype=object)

In [0]:
all_data['_P_emaildomain__addr1'] = all_data['P_emaildomain'] + '__' + all_data['addr1'].astype(str)
all_data['_card1__card2'] = all_data['card1'].astype(str) + '__' + all_data['card2'].astype(str)
all_data['_card1__addr1'] = all_data['card1'].astype(str) + '__' + all_data['addr1'].astype(str)
all_data['_card2__addr1'] = all_data['card2'].astype(str) + '__' + all_data['addr1'].astype(str)
all_data['_card12__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
all_data['_card_all__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
# Turn all of these into fractions instead of categories. So continuous numbers

In [0]:
# all_data['_amount_decimal'] = ((all_data['TransactionAmt'] - all_data['TransactionAmt'].astype(int)) * 1000).astype(int)
# all_data['_amount_decimal_len'] = all_data['TransactionAmt'].apply(lambda x: len(re.sub('0+$', '', str(x)).split('.')[1]))
# all_data['_amount_fraction'] = all_data['TransactionAmt'].apply(lambda x: float('0.'+re.sub('^[0-9]|\.|0+$', '', str(x))))
# #all_data[['TransactionAmt','_amount_decimal','_amount_decimal_len','_amount_fraction']].head(10)

In [0]:
# all_data['TransactionAmt_to_mean_card1'] = all_data['TransactionAmt'] / all_data.groupby(['card1'])['TransactionAmt'].transform('mean')
# all_data['TransactionAmt_to_mean_card4'] = all_data['TransactionAmt'] / all_data.groupby(['card4'])['TransactionAmt'].transform('mean')
# all_data['TransactionAmt_to_std_card1'] = all_data['TransactionAmt'] / all_data.groupby(['card1'])['TransactionAmt'].transform('std')
# all_data['TransactionAmt_to_std_card4'] = all_data['TransactionAmt'] / all_data.groupby(['card4'])['TransactionAmt'].transform('std')

# all_data['id_02_to_mean_card1'] = all_data['id_02'] / all_data.groupby(['card1'])['id_02'].transform('mean')
# all_data['id_02_to_mean_card4'] = all_data['id_02'] / all_data.groupby(['card4'])['id_02'].transform('mean')
# all_data['id_02_to_std_card1'] = all_data['id_02'] / all_data.groupby(['card1'])['id_02'].transform('std')
# all_data['id_02_to_std_card4'] = all_data['id_02'] / all_data.groupby(['card4'])['id_02'].transform('std')

# all_data['D15_to_mean_card1'] = all_data['D15'] / all_data.groupby(['card1'])['D15'].transform('mean')
# all_data['D15_to_mean_card4'] = all_data['D15'] / all_data.groupby(['card4'])['D15'].transform('mean')
# all_data['D15_to_std_card1'] = all_data['D15'] / all_data.groupby(['card1'])['D15'].transform('std')
# all_data['D15_to_std_card4'] = all_data['D15'] / all_data.groupby(['card4'])['D15'].transform('std')

# all_data['D15_to_mean_addr1'] = all_data['D15'] / all_data.groupby(['addr1'])['D15'].transform('mean')
# all_data['D15_to_mean_addr2'] = all_data['D15'] / all_data.groupby(['addr2'])['D15'].transform('mean')
# all_data['D15_to_std_addr1'] = all_data['D15'] / all_data.groupby(['addr1'])['D15'].transform('std')
# all_data['D15_to_std_addr2'] = all_data['D15'] / all_data.groupby(['addr2'])['D15'].transform('std')

In [58]:

cat_cols = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2',
            'P_emaildomain','R_emaildomain',
             'M1','M2','M3','M4','M5','M6','M7','M8', '_hour',
            'M9','DeviceType','DeviceInfo'] + [f'id_{i}' for i in range(12,39)]



for i, t in df.loc[:, df.columns != 'isFraud'].dtypes.iteritems():
    if t == object:
        cat_cols.append(i)
        all_data[i] = all_data[i].astype(str)
        all_data[i].fillna('unknown', inplace=True)
        #all_data[i] = pd.factorize(all_data[i])[0]
        #all_data[i] = all_data[i].astype('category')
cat_cols = list(set(cat_cols))
print(cat_cols)

enc_cols = []
for i, t in df.loc[:, df.columns != 'isFraud'].dtypes.iteritems():
    if t == object:
        cat_cols.append(i)
        enc_cols.append(i)
        all_data[f'{i}__fac'] = pd.factorize(all_data[i])[0]
        #all_data[i] = all_data[i].astype('category')
print(enc_cols)
cat_cols = list(set(cat_cols))

['M3', 'M5', 'M6', 'id_13', 'addr1', '_card12__addr1', 'id_15', 'M1', 'P_emaildomain', 'id_23', 'id_30', 'DeviceType', 'id_36', '_hour', 'M9', 'id_18', 'id_37', 'M8', 'id_25', 'id_38', '_card_all__addr1', 'card6', 'DeviceInfo', 'id_12', 'id_24', 'R_emaildomain', 'id_32', 'id_35', 'id_27', 'id_26', 'id_33', '_weekday', 'id_21', '_card2__addr1', 'M7', 'id_16', 'card4', 'addr2', 'id_28', 'card3', '_card1__card2', 'card1', 'card2', 'M4', 'M2', 'id_17', 'id_31', '_card1__addr1', '_P_emaildomain__addr1', 'id_19', 'ProductCD', 'id_34', 'id_14', 'card5', 'id_20', 'id_22', 'id_29']
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', '_weekday', '_hour', '_P_emaildomain__addr1', '_card1__card2', '_card1__addr1', '_card2__addr1', '_card12__addr1', '_card_all__addr1']


In [0]:
# def clean_inf_nan(df):
#     return df.replace([np.inf, -np.inf], np.nan) 
# df = clean_inf_nan(df)
# for col in df.columns:
#   if col in cat_cols:
#     df[col].fillna('NaN')


In [0]:
dep_var = 'isFraud'
cont_cols = list(set(all_data.columns)-set(cat_cols)-{dep_var})
#Following SafeBox Kernel and dropping highly correlated features
drop_col = ['V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
for c in drop_col:
    if c in cat_cols:
        cat_cols.remove(c)
    else:
        cont_cols.remove(c)

In [0]:
# embed_cat_cols = {}
# for col in cat_cols:
#     embed_cat_cols[col] = 10

In [0]:
# all_data = df

In [0]:
df = all_data
#df_test = all_data[all_data['isFraud'].isnull()].drop('isFraud', axis=1)
#Y_train = X_train.pop('isFraud')
del all_data

In [0]:
# !git clone https://github.com/sdoria/mish 

In [0]:
# #Functions
# %cd mish
# from rangerlars import *
# from mish import *
# from mxresnet import *
# from ranger import *

In [0]:
procs = [FillMissing, Categorify, Normalize] # Order matters
path = '';
metrics = [AUROC(), accuracy]
label_cls = CategoryList
y_range = None
dep_var = 'isFraud'

In [0]:
bs=4096
split_idx=int(0.9*df.shape[0])
# Take one sample of NA for each column
na_idxs = np.unique(np.concatenate([np.where(df[k].isna())[0][0:1] for k in df.keys()]))
train_idxs = np.concatenate([
    np.arange(split_idx),
    na_idxs[np.where(na_idxs>split_idx)],
])
valid_idxs = np.setdiff1d(arange_of(df), train_idxs)
# train_idxs.shape, valid_idxs.shape
# data = TabularList.from_df(df, path=path, procs=procs, cont_names=cont_cols, cat_names=cat_cols) \
#     .split_by_idxs(train_idxs, valid_idxs) \
#     .label_from_df(cols=dep_var, label_cls=label_cls, **kwargs).databunch(bs=bs)

#valid_idx = range(int(np.ceil(len(df)-0.2*len(df))), len(df))
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idxs, procs=procs, cat_names=cat_cols, 
                               bs=bs) 

In [0]:
len(data.train_ds), len(data.valid_ds)

In [0]:
#act_fn = Mish()
#opt_func = partial(Ranger, betas=(0.9,0.99), eps=1e-6)
from torch.nn.modules.loss import NLLLoss, CrossEntropyLoss

In [0]:
# learn = tabular_learner(data_bunch, layers=[200, 100], ps=[0.001, 0.01], 
#                         emb_drop=0.04, metrics=metrics, y_range=y_range, emb_szs=embed_cat_cols)
#                        #opt_func=opt_func)
learn = tabular_learner(data, layers=[200, 100], #emb_szs=embed_cat_cols,
               bn_wd=False, true_wd=True, metrics=metrics, emb_drop=0.04,
               loss_func=LabelSmoothingCrossEntropy())# ps=[0.001, 0.01])
               #loss_func=LabelSmoothingCrossEntropy())
                # opt_func=opt_func)
                #loss_func=LabelSmoothingCrossEntropy(),

In [0]:
learn

In [0]:
learn.lr_find(stop_div=True, num_it=100)
learn.recorder.plot()


In [47]:
# experiment = Experiment(api_key="mvOjAWu7j7F4yvKipHGocF3ew", project_name="ieee", workspace="liamlio")
# with experiment.train():
learn.fit_one_cycle(3)
# learn.fit_one_cycle(1, 1e-2, wd=0.2)
# learn.fit_one_cycle(3, 1e-3, wd=0.2)


epoch,train_loss,valid_loss,auroc,accuracy,time
0,0.323278,0.313103,0.876279,0.969790,01:11
1,0.238557,0.270560,0.877601,0.972618,01:11
2,0.224354,0.263005,0.879103,0.972161,01:10


In [48]:


from fastai.callbacks.tracker import EarlyStoppingCallback
cbs=[EarlyStoppingCallback(learn, monitor='auroc', min_delta=0.001, patience=3, )]

learn.fit_one_cycle(30, 1e-4, wd=0.2, callbacks=cbs)

epoch,train_loss,valid_loss,auroc,accuracy,time
0,0.220487,0.265357,0.876123,0.971890,01:10
1,0.220314,0.262742,0.877519,0.971873,01:10
2,0.220073,0.265694,0.874343,0.971822,01:10
3,0.219517,0.267629,0.871366,0.972093,01:10
4,0.219385,0.269624,0.877603,0.971992,01:11


Epoch 5: early stopping


In [0]:
filename='ieee12'
#!mkdir models
learn.export(f'models/{filename}.pkl')

In [50]:
# Cleanup

del learn
#del df
import gc
del df
gc.collect()

23

# Test

In [51]:
%cd /content
df_test = load_df('test')

/content


In [0]:
df_test[dep_var] = 0

In [0]:
all_data = df_test

In [0]:
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
all_data['Date'] = all_data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
all_data['_weekday'] = all_data['Date'].dt.dayofweek
all_data['_hour'] = all_data['Date'].dt.hour
all_data['_day'] = all_data['Date'].dt.day

all_data['_weekday'] = all_data['_weekday'].astype(str)
all_data['_hour'] = all_data['_hour'].astype(str)
all_data['_weekday__hour'] = all_data['_weekday'] + all_data['_hour']

cnt_day = all_data['_day'].value_counts()
cnt_day = cnt_day / cnt_day.mean()
all_data['_count_rate'] = all_data['_day'].map(cnt_day.to_dict())

all_data.drop(['TransactionDT','Date','_day'], axis=1, inplace=True)

In [0]:
all_data['_P_emaildomain__addr1'] = all_data['P_emaildomain'] + '__' + all_data['addr1'].astype(str)
all_data['_card1__card2'] = all_data['card1'].astype(str) + '__' + all_data['card2'].astype(str)
all_data['_card1__addr1'] = all_data['card1'].astype(str) + '__' + all_data['addr1'].astype(str)
all_data['_card2__addr1'] = all_data['card2'].astype(str) + '__' + all_data['addr1'].astype(str)
all_data['_card12__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
all_data['_card_all__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
# Turn all of these into fractions instead of categories. So continuous numbers

In [0]:
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2',
            'P_emaildomain','R_emaildomain',
             'M1','M2','M3','M4','M5','M6','M7','M8', '_hour',
            'M9','DeviceType','DeviceInfo'] + [f'id_{i}' for i in range(12,39)]



for i, t in df.loc[:, df.columns != 'isFraud'].dtypes.iteritems():
    if t == object:
        cat_cols.append(i)
        all_data[i] = all_data[i].astype(str)
        all_data[i].fillna('unknown', inplace=True)
        #all_data[i] = pd.factorize(all_data[i])[0]
        #all_data[i] = all_data[i].astype('category')
cat_cols = list(set(cat_cols))
print(cat_cols)

enc_cols = []
for i, t in df.loc[:, df.columns != 'isFraud'].dtypes.iteritems():
    if t == object:
        cat_cols.append(i)
        enc_cols.append(i)
        all_data[f'{i}__fac'] = pd.factorize(all_data[i])[0]
        #all_data[i] = all_data[i].astype('category')
print(enc_cols)
cat_cols = list(set(cat_cols))

In [0]:
dep_var = 'isFraud'
cont_cols = list(set(all_data.columns)-set(cat_cols)-{dep_var})
#Following SafeBox Kernel and dropping highly correlated features
drop_col = ['V300','V309','V111','V124','V106','V125','V315','V134','V102','V123','V316','V113',
              'V136','V305','V110','V299','V289','V286','V318','V304','V116','V284','V293',
              'V137','V295','V301','V104','V311','V115','V109','V119','V321','V114','V133','V122','V319',
              'V105','V112','V118','V117','V121','V108','V135','V320','V303','V297','V120',
              'V1','V14','V41','V65','V88', 'V89', 'V107', 'V68', 'V28', 'V27', 'V29', 'V241','V269',
              'V240', 'V325', 'V138', 'V154', 'V153', 'V330', 'V142', 'V195', 'V302', 'V328', 'V327', 
              'V198', 'V196', 'V155']
for c in drop_col:
    if c in cat_cols:
        cat_cols.remove(c)
    else:
        cont_cols.remove(c)

In [0]:
df_test = all_data
del all_data

In [0]:
filename='ieee12'

In [80]:
tr = pd.isnull(df_test['C13'])
df_test[tr]= -1
pd.isnull(df_test['C13']).sum(0)

0

In [0]:
learn = load_learner(
    '',
    f'models/{filename}.pkl', 
    TabularList.from_df(df_test, path=path, cat_names=cat_cols, cont_names=cont_cols, procs=procs),
    bs=bs
)

In [0]:
test_preds, targs = learn.get_preds(ds_type=DatasetType.Test)

In [52]:
len(np.where(test_preds[:, 1]>0.5)[0])

0

In [0]:
# Sanity check
assert(len(np.where(test_preds[:, 1]>0.5)[0])>8000)

In [0]:
submission = df_test[['TransactionID']].copy()
submission['isFraud']=test_preds[:, 1]

In [0]:
submission.head()

In [0]:
name = filename
submission.to_csv('submission.csv', index=False, header=True)

In [58]:
from IPython.display import FileLink
FileLink(r'submission.csv')

/content/submission.csv